In [65]:
import pandas as pd
import numpy as np
import datetime

#read in dataset using pandas
AIV = pd.read_csv('../input/dataenergy/B1780.csv') # Aggregated Imbalance Volumes
IP = pd.read_csv("../input/dataenergy/B1770.csv") # Imbalance Prices


from datetime import datetime
def settle_time(settle_fig):
    """function to add time series half hour period to datasets"""
    
    time_data = settle_fig["Settlement Date"][0] #collect first date from table 
    day = time_data[:2] #split date given to use to make datetime format
    print(day)
    month = time_data[3:5]
    year = time_data[6:10]
    start = datetime(int(year), int(month), int(day), 0, 0)
    half_hour = pd.Timedelta('00:30:00') #half an hour value
    start_value = pd.Timedelta('00:00:00')


    for i in range(len(settle_fig)):
        """range based for loop for setting date time values in dataset"""
        
        settle_fig["Settlement Date"][i] = start + start_value
        start_value += half_hour
    
    settle_fig["Settlement Date"] = settle_fig["Settlement Date"].values[::-1] #use to reverse datetime values so it matches settlement period

def settle_time_half_hourly(settle_fig):
    """function to add time series half hour period to datasets"""
    
    time_data = settle_fig["Settlement Date"][0] #collect first date from table 
    day = time_data[:2] #split date given to use to make datetime format
    print(day)
    month = time_data[3:5]
    year = time_data[6:10]
    start = datetime(int(year), int(month), int(day), 0, 0)
    half_hour = pd.Timedelta('00:30:00') #half an hour value
    start_value = pd.Timedelta('00:00:00')


    for i in range(len(settle_fig)):
        """range based for loop for setting date time values in dataset"""
        
        settle_fig["Settlement Date"][i] = start + start_value
        start_value += half_hour
    
    settle_fig["Settlement Date"] = settle_fig["Settlement Date"].values[::-1] #use to reverse datetime values so it matches settlement period
        
        
#         #print(settle_fig["Settlement Date"])
#     for i in settle_fig["Settlement Date"]:
#         settle_fig["Settlement Date"][i] = start + start_value
#         start_value =+  half_hour
        
def settle_time(settle_fig):
    """function to add time series half hour period to datasets"""
    
    time_data = settle_fig["Settlement Date"][0] #collect first date from table 
    day = time_data[:2] #split date given to use to make datetime format
    #print(day)
    month = time_data[3:5]
    year = time_data[6:10]
    start = datetime(int(year), int(month), int(day), 0, 0)
    half_hour = pd.Timedelta('00:30:00') #half an hour value
    start_value = pd.Timedelta('00:00:00')


    for i in range(len(settle_fig)):
        """range based for loop for setting date time values in dataset"""
        
        settle_fig["Settlement Date"][i] = start + start_value
        start_value += half_hour
    
    settle_fig["Settlement Date"] = settle_fig["Settlement Date"].values[::-1] #use to reverse datetime values so it matches settlement period

def settle_time_quarter_hourly(settle_fig):
    """function to add time series half hour period to datasets"""
    """Due to repeating 'Settlement period', values seperated by quarter of an hour"""
    
    time_data = settle_fig["Settlement Date"][0] #collect first date from table 
    day = time_data[:2] #split date given to use to make datetime format
    #print(day)
    month = time_data[3:5]
    year = time_data[6:10]
    start = datetime(int(year), int(month), int(day), 0, 0)
    hour = pd.Timedelta('00:15:00') #half an hour value
    start_value = pd.Timedelta('00:00:00')


    for i in range(len(settle_fig)):
        """range based for loop for setting date time values in dataset"""
        
        settle_fig["Settlement Date"][i] = start + start_value
        start_value += hour
    
    settle_fig["Settlement Date"] = settle_fig["Settlement Date"].values[::-1] #use to reverse datetime values so it matches settlement period



def total_imbalance_cost(dataset):
    """Generator to calculate the total daily imbalance cost"""
    """Values are repeated due to 'Settlement Period'"""
    """Also being repated, sum of column has to be divided by two"""
    
    total = dataset['Imbalance Price Amount (GBP)'].sum() / 2
    
    
    
    print(f'The total imbalance cost for day ending at {dataset["Settlement Date"][0]} is £{total} (GBP)' )
    
def daily_imbalance_unit_rate(dataset):
    """Generator to calculate imbalance unit rate"""
    """Values given are set in MWh"""
    """Need to be converted to MW per Hour or MW/h"""
    """Calculation is MWh/h^2"""
    
    total_imbalance = dataset['Imbalance Quantity (MWh)'].sum()
    hours = str(dataset['Settlement Date'][0])
    hours = hours[10:]
    hours = pd.Timedelta(hours)
    hours = hours.total_seconds()
    hours = hours / (60*60) #find total amount of hours in the trading day
    
    imbalance_rate = total_imbalance / (hours*hours)
    
    print(f'The imbalance unite rate in MegaWatts per hour for day ending at {dataset["Settlement Date"][0]} is {imbalance_rate} MW/h' )
    
    
    
    
settle_time_half_hourly(AIV)
settle_time_quarter_hourly(IP) #repeating 'Settlement Period', time series added for quarter hour
display(AIV)
display(IP)
total_imbalance_cost(IP)
daily_imbalance_unit_rate(AIV)

In [66]:
#DATA LOADING

# ip_time_series.loc[:, 'Settlement Date':'Imbalance Price Amount (GBP)']
# display(ip_time_series)

def reset_my_index(dataset):
    """function for removing old index"""
    res = dataset[::-1] .reset_index(drop=True)
    return(res) 

ip_cols = ['Settlement Date', 'Imbalance Price Amount (GBP)']
ip_time_series = pd.DataFrame(IP, columns=ip_cols)
ip_time_series = reset_my_index(ip_time_series)

aiv_cols = ['Settlement Date', 'Imbalance Quantity (MWh)']
aiv_time_series = pd.DataFrame(AIV, columns=aiv_cols)
aiv_time_series = reset_my_index(aiv_time_series)


display(ip_time_series)
display(aiv_time_series)

#DOWNSAMPLING IMBALANCES PRICES FOR 15MINS TO HALF HOUR
#REDUCE FREQUENCY OF DATA TO 30 MINS

ip_time_series = ip_time_series.resample('0.5H', on = 'Settlement Date').mean()
ip_time_series.tail()


In [67]:
#VISUALISATION FOR AGGREGATE INBALANCE VOLUME AND IMBALANCE PRICES

#ip_time_series.index = ip_time_series['Settlement Date'] #changing index to date, for showing on plot
#No need to change index of IP as it is already changed
ip_time_series['Imbalance Price Amount (GBP)'].plot()


aiv_time_series.index = aiv_time_series['Settlement Date'] #changing index to date, for showing on plot
aiv_time_series['Imbalance Quantity (MWh)'].plot()




In [68]:
#VISUALISATION FOR INBALANCE PRICES

ip_time_series['Imbalance Price Amount (GBP)'].plot()

#zoom into hour with the highest peak
print(f"The highest Imbalancce Quantity is £{ip_time_series['Imbalance Price Amount (GBP)'].max()} at {ip_time_series['Imbalance Price Amount (GBP)'].idxmax()}")

In [69]:
#VISUALISATION FOR INBALANCE PRICES
aiv_time_series['Imbalance Quantity (MWh)'].plot()

print(f"The highest Imbalancce Price amount is {aiv_time_series['Imbalance Quantity (MWh)'].max()}MWh at {aiv_time_series['Imbalance Quantity (MWh)'].idxmax()}")

**VALIDATE WITH NAIVE AND ARIMA MODEL**

In [70]:
# CREATE NAIVE PERSISTENCE MODEL 
# IF ARIMA MODEL CANNOT IMPORVE UPON MEAN SQUARED ERROR
# IT MEANS TIME SERIES IS A WALK

#NAIVE FORECAST 

aiv_time_series['t'] = aiv_time_series['Imbalance Quantity (MWh)'].shift(1) #walk forward shift by one
ip_time_series['t'] = ip_time_series['Imbalance Price Amount (GBP)'].shift(1)

aiv_train, aiv_test = aiv_time_series[1:aiv_time_series.shape[0]-7], aiv_time_series[aiv_time_series.shape[0]-7:] #split data into test and training
ip_train, ip_test = ip_time_series[1:ip_time_series.shape[0]-7], ip_time_series[ip_time_series.shape[0]-7:] #split data into test and training
#take last seven values as test data
#first data not taken since it becomes Null/NaN

#AIV train and test split
aiv_train_X, aiv_train_y = aiv_train['t'], aiv_train['Imbalance Quantity (MWh)']
aiv_test_X, aiv_test_y = aiv_test['t'], aiv_test['Imbalance Quantity (MWh)']

#IP train and test split
ip_train_X, ip_train_y = ip_train['t'], ip_train['Imbalance Price Amount (GBP)']
ip_test_X, ip_test_y = ip_test['t'], ip_test['Imbalance Price Amount (GBP)']

#WALK FORWARD VALIDADTION 
aiv_predictions = aiv_test_X.copy()
ip_predictions = ip_test_X.copy()

from sklearn.metrics import mean_squared_error
aiv_mse = mean_squared_error(aiv_test_y, aiv_predictions)
ip_mse = mean_squared_error(ip_test_y, ip_predictions)

print(f'AIV mean squared error: {aiv_mse}, IP mean squared error: {ip_mse}') # test later to see if ARIMA model can improve upon this
#helps evaluate advanced models, if advanced movels gives value greater than this: we cannot extract any imformation from this data
#therefore data would be considered a random walk



In [71]:
from matplotlib import pyplot
pyplot.plot(aiv_test_y)
pyplot.plot(aiv_predictions, color = 'red') # red line is predicted value, similar trajectory to actual value

In [72]:
pyplot.plot(ip_test_y)
pyplot.plot(ip_predictions, color = 'red') # red line represents predicted value using current data

In [73]:
#CREATE AN ARIMA MODEL
#Our data has no seasonality 
#Data trend appears to be linear, therefore D=1
#autocorrelation plot

from pandas.plotting import autocorrelation_plot
autocorrelation_plot(aiv_time_series['Imbalance Quantity (MWh)'], color = 'red') #AIV in red
autocorrelation_plot(ip_time_series['Imbalance Price Amount (GBP)'], color = 'blue') #IP in blue

#confidence interval are the bounds (straight lines)
#both crosses confidence interval at around one
#Consider 1 as the value for p
#p = 1


In [74]:
#partial autocorrelation

from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(aiv_time_series['Imbalance Quantity (MWh)'], lags = 15)
#crosses confidence interval at one
#q =2


In [75]:
plot_pacf(ip_time_series['Imbalance Price Amount (GBP)'], lags = 15)
#also crosses confidence interval at 1
#q = 2

In [76]:
#Arima model for AIV
from statsmodels.tsa.arima.model import ARIMA
model_aiv = ARIMA(aiv_time_series['Imbalance Quantity (MWh)'], order=(1,1,1))
model_aiv_fit = model_aiv.fit()
model_aiv_fit.summary()


In [77]:
residuals_aiv = model_aiv_fit.resid
residuals_aiv.plot()
residuals_aiv.describe()
#residual should be white noise: no apparant pattern
# good sign as pattern means model can be improved and has information missing
#no trend or seasonality apparent


In [78]:
#Forecast agregated imbalance volume for a following few Hours
output_aiv = model_aiv_fit.forecast(5)
output_aiv

In [79]:
#ARIMA model for IP
model_ip = ARIMA(ip_time_series['Imbalance Price Amount (GBP)'], order=(1,1,1))
model_ip_fit = model_aiv.fit()
model_ip_fit.summary()

In [80]:
residuals_ip = model_ip_fit.resid
residuals_ip.plot()
residuals_ip.describe()

In [81]:
#Forecast imbalance prices for a following few Hours
output_ip = model_ip_fit.forecast(5)
output_ip

In [82]:
#EVALUATING THE RELATIONSHIP BETWEEN AGGREGATED IMBALANCE VOLUMES AND IMBALANCE PRICE
#compare percentage changes rather than simply correlation
#percentage changes give a better understanding but unlikely in this scenario
import matplotlib.pyplot as plt

ip_time_series['IP_ret'] = ip_time_series['Imbalance Price Amount (GBP)'].pct_change()
aiv_time_series['AIV_ret'] = aiv_time_series['Imbalance Quantity (MWh)'].pct_change()

#visualisation and correlation
plt.scatter(ip_time_series['Imbalance Price Amount (GBP)'], aiv_time_series['Imbalance Quantity (MWh)'])
plt.show()
correlation = ip_time_series['Imbalance Price Amount (GBP)'].corr(aiv_time_series['Imbalance Quantity (MWh)'])
print("The correlation without percentage change is: ", correlation)
#Fairly strong positive correlation shown



In [83]:
plt.scatter(ip_time_series['IP_ret'], aiv_time_series['AIV_ret'])
plt.show()
correlation = ip_time_series['IP_ret'].corr(aiv_time_series['AIV_ret'])
print("The correlation with percentage change is: ", correlation)
#No correlation observed whatsover with this